In [16]:
import hoomd,imp
import hoomd
from hoomd import *
from hoomd import md
import numpy as np
import gsd.hoomd
import time 
import matplotlib.pyplot as plt
from decimal import *
from inspect import currentframe
import string

# class SimBox(self):
class SimBox():
#   def __init__(self, boxsize,nparticles):
#        self.boxsize=boxsize
#        self.nparticles=nparticles
        
    
    ###Creates Initial Snapshot
    def MakeSnapshot(self, boxsize, nparticles): 
        
        snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(L=boxsize+nparticles),
                                          bond_types=['tether'],particle_types=self.GenParticleTypes(nparticles, debug=True))
        return snapshot
    
    
    #Only used for initializing the system, do not use outside of MakeSnapshot!
    def GenParticleTypes(self, nparticles, debug=None):
        counter=0
        #List that will contain the particle types. Enable debug to print.
        GPT = [] 
        for k in range(nparticles):
            counter=k
            GPT.append('C%d'%counter)
        if(debug==True):
            print("DEBUG: GenParticleTypes: Particle Types = ",GPT)
        return GPT

class Polymer:

    def SetPolyProperties(self, nparticles, boxsize, diam=None, debug=None):
        if(diam==None):
            diam=1.0 #default diameter
        for i in range(nparticles):
            system.particles.get(i).position = [-(boxsize-nparticles)+(((boxsize-nparticles))/nparticles)*i,0,0]
            system.particles.get(i).diameter = 1.0
            #system.particles.get(i).type = ('C%d'%(i))
            if(debug==True):
                print("DEBUG: SetPolyProperties: Particle %d is at "%i, [nparticles+i,0,0])
    
    #MakePolyTage is just a debugging tool, ignore. 
    def MakePolyTags(self, nparticles, debug=None):
        P_Tag_List = []
        for i in range(nparticles):
            system.particles.get(i).type=('C%d'%(i+1))
            P_Tag_List.append('C%d'%(i+1))
        if(debug==True):
            print("DEBUG: SetPolyProperties: P_Tag_List is ", P_Tag_List)
        
    def PatchTypes(self, tot_npatch, debug=None):
        patch_types = []
        for i in range(tot_npatch):
            system.particles.types.add('%d'%(i+1))
            patch_types.append('%d'%(i+1))
        if(debug==True):
            print("DEBUG: PatchTypes: ",patch_types)
        return patch_types

            
    
    def MakeBonds(self, nparticles, debug=None):
        DebugBonds = []
        f=0
        for f in range(nparticles-1):
            system.bonds.add('tether',f,f+1)
            DebugBonds.append([f,f+1])
            f+=1
        if(debug==True):
            print("DEBUG: MakeBonds: Bonds = ", DebugBonds)  
class Debug:
    def PrintAllParticleTags(self, nparticles, tot_npatch):
        X = []
        for i in range(nparticles+tot_npatch):
            X.append(system.particles[i].tag)
        print("DEBUG: List of All Particle Tags: ", X)
    def PrintSysParticles(self, debug=None):
        if(debug==True):
            for i in range(len(system.particles)):
                print(system.particles[i])
    def PrintLJPairs(self, sig, types):
        for i in range(0,len(sig)):
            for j in range(i,len(sig)):
                print([types[i],types[j]])
    def DebugMessage(self, string, variable, debug=None):
        if(debug==True):
            cf = currentframe()
            print("%s"%string, variable, " at line ", cf.f_back.f_lineno)
    def DebugInfo(self):
        print("Debug Message Format Is As Follows:\n DEBUG: Function: Parameter = Value at line <linenumber>")

class MomentOfInertia:
  
    #Principle Moment Calculator
    #added an option to return the inertia tensor for debugging
    def Moment(self, coord, Mass, return_tensor = None, debug=None):
        DB = Debug()
        #Referencing local variables
        cx,cy,cz = 0.0,0.0,0.0
        x,y,z = 0.0,0.0,0.0
        cXYZ = []
        Ixx,Ixy,Ixz,Iyy,Iyz,Izz = 0.0,0.0,0.0,0.0,0.0,0.0
        TotalMass = sum(Mass)
        if(debug==True):
            print("DEBUG: Moment: len(coord)= ", len(coord))
            print("DEBUG: Moment: TotalMass = ", TotalMass)
        for i in range(len(Mass)):
                x += Mass[i]*coord[i][0]
                y += Mass[i]*coord[i][1]
                z += Mass[i]*coord[i][2]
                if(debug==True):
                    print("DEBUG: Moment: x = Mass[%d]"%i +"*coord[%d][0]"%i +" =%f "%x)
                    print("DEBUG: Moment: y = Mass[%d]"%i +"*coord[%d][1]"%i +" =%f "%y)
                    print("DEBUG: Moment: z = Mass[%d]"%i +"*coord[%d][2]"%i +" =%f "%z)
        cx = x/TotalMass
        cy = y/TotalMass
        cz = z/TotalMass
        if(debug==True):
            print("DEBUG: Moment: cx = x/TotalMass = ", cx)
            print("DEBUG: Moment: cy = y/TotalMass = ", cy)
            print("DEBUG: Moment: cz = y/TotalMass = ", cz)
        com = [cx,cy,cz]
        for i in range(len(coord)):
            cXYZ.append([coord[i][0]-com[0], coord[i][1]-com[1], coord[i][2]-com[2]]) 
        if(debug==True):
            print("DEBUG: Moment: cXYZ = ", cXYZ)
            print("DEBUG: Moment: len(cXYZ) = ", len(cXYZ))
        #Constructs Inertia Tensor
        #Useful Property: Ixy=Iyx, Ixz=Izx, Iyz=Izy
        for i in range(len(coord)):
            Ixx += Mass[i]*(cXYZ[i][1]**2 + cXYZ[i][2]**2)
            Iyy += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][2]**2)
            Izz += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][1]**2)
            Ixy += -Mass[i]*cXYZ[i][0]*cXYZ[i][1]
            Ixz += -Mass[i]*cXYZ[i][0]*cXYZ[i][2]
            Iyz += -Mass[i]*cXYZ[i][1]*cXYZ[i][2]
        if(debug==True):
            print("DEBUG: Moment: Ixx = ", Ixx)
            print("DEBUG: Moment: Iyy = ", Iyy)
            print("DEBUG: Moment: Izz = ", Izz)
            print("DEBUG: Moment: Ixy = ", Ixy)
            print("DEBUG: Moment: Ixz = ", Ixz)
            print("DEBUG: Moment: Iyz = ", Iyz)

        #Computes eignevalues, finds principle moments
        Imatrix = np.matrix([[Ixx,Ixy,Ixz],[Ixy,Iyy,Iyz],[Ixz,Iyz,Izz]])
        Idiag = np.linalg.eig(Imatrix)
        if(debug==True):
            print("DEBUG: Moment: Idiag = ",Idiag)
            print("DEBUG: Moment: Idiag[0] = ",Idiag[0])
        comX = 0-com[0]
        comY = 0-com[1] 
        for i in range(len(coord)):
            coord[i] = [coord[i][0]+comX,coord[i][1]+comY,coord[i][2]]
        if(return_tensor==True):
            return Idiag[0], Imatrix
        else:
            return Idiag[0]
#Just for flexability 
def Angle(angle):
    angle = angle*np.pi/180
    return angle
DB = Debug()
DB.DebugInfo()
SB = SimBox()
MoI = MomentOfInertia()
Poly = Polymer()
#Filename (date and time will be appended onto the file name)
filename = "UBQLN2"
#Box Size
boxsize = 5
#Number of constituent particles
nparticles = 177 #will need to be 1
#Total number of patches in sim
#tot_npatch = 79
tot_npatch = 4
#Rigid Diameter
diam = 1.0
#List of Patch Diameters
pdiam = [1.0]
#LJ Cutoff
rcut = 2.5
#Random Seed
SEED = np.random.randint(0,100000000)
#Empty list for setting up sig values
Sigma = []
#Empty list for setting up eps values
Epsilon = []
#Empty list for yukawa kappa
YukKap = []
#Empty list for yukawa eps
YukEps = []
#Allows me to make unique file names
t = time.strftime(time.strftime("%d %b %H:%M:%S", time.gmtime()))
#Harmonic Bond Length
hbl = .380
#Temperature/ kT value
Temperature=200
kTinput=Temperature * 8.3144598/1000.
##----------SETUP----------##
context.initialize("")
amino_masses = dict([['A',89.094],['C',121.150],['D',133.103],['E',147.130],['F',165.192],['G',75.067],
                    ['H',155.157],['I',131.175],['K',146.19],['L',131.175],['M',149.210],['N',132.119],
                    ['P',115.132],['Q',146.146],['R',174.204],['S',105.093],['T',119.120],['V',117.148],
                    ['W',204.229],['Y',181.191],
                    ['a',89.094],['c',121.150],['d',133.103],['e',147.130],['f',165.192],['g',75.067],
                    ['h',155.157],['i',131.175],['k',146.19],['l',131.175],['m',149.210],['n',132.119],
                    ['p',115.132],['q',146.146],['r',174.204],['s',105.093],['t',119.120],['v',117.148],
                    ['w',204.229],['y',181.191]])

#From source [4]
amino_diam = dict([['A',5.04],['C',5.48],['D',5.58],['E',5.92],['F',6.36],['G',4.50],
                    ['H',6.08],['I',6.18],['K',6.36],['L',6.18],['M',6.18],['N',5.68],
                    ['P',5.56],['Q',6.02],['R',6.56],['S',5.18],['T',5.62],['V',5.86],
                    ['W',6.78],['Y',6.46],['a',5.04],['c',5.48],['d',5.58],['e',5.92],['f',6.36],['g',4.50],
                    ['h',6.08],['i',6.18],['k',6.36],['l',6.18],['m',6.18],['n',5.68],
                    ['p',5.56],['q',6.02],['r',6.56],['s',5.18],['t',5.62],['v',5.86],
                    ['w',6.78],['y',6.46]])
# amino_diam = dict([['A',0.504],['C',0.548],['D',0.558],['E',0.592],['F',0.636],['G',0.450],
#                     ['H',0.608],['I',0.618],['K',0.636],['L',0.618],['M',0.618],['N',0.568],
#                     ['P',0.556],['Q',0.602],['R',0.656],['S',0.518],['T',0.562],['V',0.586],
#                     ['W',0.678],['Y',0.646],['a',0.504],['c',0.548],['d',0.558],['e',0.592],['f',0.636],['g',0.450],
#                     ['h',0.608],['i',0.618],['k',0.636],['l',0.618],['m',0.618],['n',0.568],
#                     ['p',0.556],['q',0.602],['r',0.656],['s',0.518],['t',0.562],['v',0.586],
#                     ['w',0.678],['y',0.646]])
amino_charge = dict([['A',0.0],['C',0.0],['D',-1.0],['E',-1.0],['F',0.0],['G',0.0],
                    ['H',0.5],['I',0.0],['K',1.0],['L',0.0],['M',0.0],['N',0.0],
                    ['P',0.0],['Q',0.0],['R',1.0],['S',0.0],['T',0.0],['V',0.0],
                    ['W',0.0],['Y',0.0],['a',0.0],['c',0.0],['d',-1.0],['e',-1.0],['f',0.0],['g',0.0],
                    ['h',0.5],['i',0.0],['k',1.0],['l',0.0],['m',0.0],['n',0.0],
                    ['p',0.0],['q',0.0],['r',1.0],['s',0.0],['t',0.0],['v',0.0],
                    ['w',0.0],['y',0.0]])

#Creates dict for patch charges
pcharge_dict = []
for i in range(tot_npatch):
    pcharge_dict.append(['{0}'.format(i+1), 0.0])
pcharge_dict = dict(pcharge_dict)

#Appends pcharge_dict to amino_charge
amino_charge.update(pcharge_dict)

##Generates all bond types and bond lengths between pairs of amino acids
seq_string = "ACDEFGHIKLMNPQRSTVWYacdefghiklmnpqrstvwy" #List of all amino acids in sim
BL = [] #Bond Lengths
BT = [] #Bond Types
for i in range(len(seq_string)):
    #for j in range(i,len(seq_string)):
    for j in range(len(seq_string)):
        BL.append(['{0}{1}'.format(seq_string[i],seq_string[j]), amino_diam[seq_string[i]]/2. + amino_diam[seq_string[j]]/2. + hbl])
        BT.append('{0}{1}'.format(seq_string[i],seq_string[j]))
BLDic = dict(BL) #Bond Length Dictionary

#Complete Sequence
#sequence = 'MRAMQalmqiqqglqtlateapglipsftpgvgvgvlgtaigpvgpvtpiGPIgpivpftPIGPIGPIGPTGPAAPPGSTGSGGPTGPTVSSAAPSETTSPTSESGPNqqfiqqmvqalaganaPQLPNPEVRFQQQLEQLNangflNREANLQALIATGGDINAAIErllgsqPSW'      
sequence = 'MrAMQALMQIQQGLQTLATEAPGLIPSfTPGVGVGVLGTAIGPVGPVTPIGPIGPIVPfTPIGPIGPIGPTGPAAPPGSTGSGGPTGPTVSSAAPSETTSPTSESGPNQQfIQQMVQALAGANAPQLPNPEVrfQQQLEQLNANGfLNrEANLQALIATGGDINAAIErLLGSQPSW'
seq_list = []
rigid_tags = []
for i in range(len(sequence)):
    seq_list.append('%s'%sequence[i])
for i in range(len(sequence)):
    for j in range(26):
        if(sequence[i] is string.ascii_lowercase[j]):
            rigid_tags.append(i)
def unique(list1): #from https://www.geeksforgeeks.org/python-get-unique-values-list/
    # intilize a null list 
    unique_list = [] 
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list


#Creates a list of all amino acid types in polymer
seq_types = unique(seq_list)



#Creates snapshot, initializes particle types
# snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(Lx=3000, Ly=27, Lz=27),
#         bond_types=['tether'],particle_types=seq_types)
snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(Lx=3000, Ly=15, Lz=15),
        bond_types=BT,particle_types=seq_types)
system = hoomd.init.read_snapshot(snapshot)

#Sets each particle's type equal to it's amino acid
for i in range(nparticles):
    system.particles.get(i).type = ('%s'%seq_list[i])
    system.particles.get(i).diameter = amino_diam['%s'%seq_list[i]]

##Adds patch Patch Types from [1,tot_npatch]
##----------Assign Masses----------##
ix=0
for p in system.particles:
    p.mass = amino_masses[p.type]
    p.charge = amino_charge[p.type]

    
    
patch_types = Poly.PatchTypes(tot_npatch, debug=False) 
types = system.particles.types
DB.DebugMessage("DEBUG: types = ", types, debug=True)

patch_sigma = kTinput
##----------Builds Dictionaries for LJ Parameters---------##
for i in range(len(types)):
    if(i>=(len(types)-tot_npatch)):
        Sigma.append(('%s'%types[i], patch_sigma))
        Epsilon.append(('%s'%types[i], 0.5)) 
    elif(i<len(types)):
        Sigma.append(('%s'%types[i], amino_diam[types[i]]))
        Epsilon.append(('%s'%types[i], 0.1))  
sig = dict(Sigma)
DB.DebugMessage("DEBUG: sig = ", sig, debug = True)
eps = dict(Epsilon)
DB.DebugMessage("DEBUG: eps = ", eps, debug = True)


##----------Builds Dictionaries for Yukawa Parameters---------##
    ##Allows independent assigning of parameters of patches and particles
# for i in range(len(types)):
#     if(i>=(len(types)-tot_npatch)): #i=npaticles is where the indices for the patch tags start
#         YukKap.append(('%s'%types[i], 0.0))
#         YukEps.append(('%s'%types[i], 0.0)) 
#     elif(i<len(types)):
#         YukKap.append(('%s'%types[i], 0.0))
#         YukEps.append(('%s'%types[i], 0.0)) 
# yukkap = dict(YukKap)
# DB.DebugMessage("DEBUG: yukkap = ", yukkap, debug = True)
# yukeps = dict(YukEps)
# DB.DebugMessage("DEBUG: yukeps = ", yukeps, debug = True)

###----------Make Rigid Bodies----------###
rigid=hoomd.md.constrain.rigid()
coord_patch_group_a = [[2.52*np.cos(Angle(60)),0,2.52*np.sin(Angle(60))],
                       [2.52*np.cos(Angle(60)),0,-2.52*np.sin(Angle(60))]]

coord_patch_group_l = [[3.09*np.cos(Angle(60)),0,3.09*np.sin(Angle(60))],
                       [3.09*np.cos(Angle(60)),0,-3.09*np.sin(Angle(60))]]

coord_patch_group_m = [[3.09*np.cos(Angle(60)),0,3.09*np.sin(Angle(60))],
                       [3.09*np.cos(Angle(60)),0,-3.09*np.sin(Angle(60))]]

coord_patch_group_q = [[3.01*np.cos(Angle(60)),0,3.01*np.sin(Angle(60))],
                       [3.01*np.cos(Angle(60)),0,-3.01*np.sin(Angle(60))]]

coord_patch_group_k = [[6.36*np.cos(Angle(60)),0,6.36*np.sin(Angle(60))],
                       [6.36*np.cos(Angle(60)),0,-6.36*np.sin(Angle(60))]]

coord_patch_group_i = [[3.09*np.cos(Angle(60)),0,3.09*np.sin(Angle(60))],
                       [3.09*np.cos(Angle(60)),0,-3.09*np.sin(Angle(60))]]

coord_patch_group_g = [[2.25*np.cos(Angle(60)),0,2.25*np.sin(Angle(60))],
                       [2.25*np.cos(Angle(60)),0,-2.25*np.sin(Angle(60))]]

coord_patch_group_t = [[2.81*np.cos(Angle(60)),0,2.81*np.sin(Angle(60))],
                       [2.81*np.cos(Angle(60)),0,-2.81*np.sin(Angle(60))]]

coord_patch_group_e = [[2.96*np.cos(Angle(60)),0,2.96*np.sin(Angle(60))],
                       [2.96*np.cos(Angle(60)),0,-2.96*np.sin(Angle(60))]]

coord_patch_group_p = [[2.78*np.cos(Angle(60)),0,2.78*np.sin(Angle(60))],
                       [2.78*np.cos(Angle(60)),0,-2.78*np.sin(Angle(60))]]

coord_patch_group_s = [[2.59*np.cos(Angle(60)),0,2.59*np.sin(Angle(60))],
                       [2.59*np.cos(Angle(60)),0,-2.59*np.sin(Angle(60))]]

coord_patch_group_f = [[3.18*np.cos(Angle(60)),0,3.18*np.sin(Angle(60))],
                       [3.18*np.cos(Angle(60)),0,-3.18*np.sin(Angle(60))]]

coord_patch_group_v = [[2.93*np.cos(Angle(60)),0,2.93*np.sin(Angle(60))],
                       [2.93*np.cos(Angle(60)),0,-2.93*np.sin(Angle(60))]]

coord_patch_group_n = [[2.84*np.cos(Angle(60)),0,2.84*np.sin(Angle(60))],
                       [2.84*np.cos(Angle(60)),0,-2.84*np.sin(Angle(60))]]

coord_patch_group_r = [[3.28*np.cos(Angle(60))/10.,0,3.28*np.sin(Angle(60))/10.],
                       [3.28*np.cos(Angle(60))/10.,0,-3.28*np.sin(Angle(60))/10.]]

rigid_neigh_1 =[[0.5*np.cos(Angle(60)),0,0.5*np.sin(Angle(60))],
                [0.5*np.cos(Angle(60)),0,-0.5*np.sin(Angle(60))],
                [-hbl,0,0],[hbl,0,0]]

# mass_patch_group_1 = [0.5,0.5,75.067,89.094]
mass_patch_groups = []
#reference rigid_tags for indices
for i in range(len(sequence)):
    for j in range(26):
        if(sequence[i] is string.ascii_lowercase[j]):
            if((i+1)>=len(sequence)):
                mass_patch_groups.append([0.5,0.5,amino_masses[sequence[i-1]]])
            elif((i+1)<len(sequence) and (i-1)>=0):
                
                mass_patch_groups.append([0.5,0.5,amino_masses[sequence[i-1]],amino_masses[sequence[i+1]]])
                
            elif((i-1)<0):
                mass_patch_groups.append([0.5,0.5,amino_masses[sequence[i+1]]])
diameters_patch = [1.0,1.0]

eigen_groups = []
for i in range(len(rigid_tags)):
    eigen_groups.append(MoI.Moment(rigid_neigh_1, mass_patch_groups[i], debug=False))


# rigid.set_param('a',positions=coord_patch_group_a, types=['1','2'], diameters = diameters_patch)
# rigid.set_param('l',positions=coord_patch_group_l, types=['3','4'], diameters = diameters_patch)
# rigid.set_param('m',positions=coord_patch_group_m, types=['5','6'], diameters = diameters_patch)
# rigid.set_param('q',positions=coord_patch_group_q, types=['7','8'], diameters = diameters_patch)
# rigid.set_param('i',positions=coord_patch_group_i, types=['9','10'], diameters = diameters_patch)
# rigid.set_param('g',positions=coord_patch_group_g, types=['11','12'], diameters = diameters_patch)
# rigid.set_param('t',positions=coord_patch_group_t, types=['13','14'], diameters = diameters_patch)
# rigid.set_param('e',positions=coord_patch_group_e, types=['15','16'], diameters = diameters_patch)
# rigid.set_param('p',positions=coord_patch_group_p, types=['17','18'], diameters = diameters_patch)
# rigid.set_param('s',positions=coord_patch_group_s, types=['19','20'], diameters = diameters_patch)
rigid.set_param('f',positions=coord_patch_group_f, types=['1','2'], diameters = diameters_patch)
# rigid.set_param('v',positions=coord_patch_group_v, types=['23','24'], diameters = diameters_patch)
# rigid.set_param('n',positions=coord_patch_group_n, types=['25','26'], diameters = diameters_patch)
rigid.set_param('r',positions=coord_patch_group_r, types=['3','4'], diameters = diameters_patch)
#rigid.set_param('k',positions=coord_patch_group_f, types=['3','4'], diameters = diameters_patch)
rigid.create_bodies()

# system.particles[191].charge = 1.00
# system.particles[192].charge = 1.00


center = hoomd.group.rigid_center()
nonrigid = hoomd.group.nonrigid()
part = hoomd.group.all()
gpoly = hoomd.group.union(name='gpoly', a=center, b=nonrigid) 
patches = hoomd.group.difference(name = 'patches', a=part, b=gpoly)
for p in patches:
    p.mass = 0.5
for i in range(len(rigid_tags)):
    Z = rigid_tags[i]
    system.particles[Z].moment_inertia = eigen_groups[i]

##----------Make Bonds----------##
bi = 0 #bond index
DebugBonds = []
for bi in range(nparticles-1):
    system.bonds.add('{0}{1}'.format(sequence[bi],sequence[bi+1]),bi,bi+1)
    #system.bonds.add('tether',bi,bi+1)
    DebugBonds.append([bi,bi+1])
    bi+=1

pos = []
for ix in range(nparticles):
    if(ix<nparticles-1):
        if(ix!=0):
            pos.append([pos[ix-1][0] + hbl + amino_diam['%s'%seq_list[ix]]/2. +amino_diam['%s'%seq_list[ix+1]]/2. ,0,0])
            #print("diams = {0} {1} {2}".format(hbl,amino_diam['%s'%seq_list[ix]],amino_diam['%s'%seq_list[ix+1]]))
        elif(ix==0):
            pos.append([-500 + hbl + amino_diam['%s'%seq_list[ix]]/2. +amino_diam['%s'%seq_list[ix+1]]/2. ,0,0])
            #print("diams = {0} {1} {2}".format(hbl,amino_diam['%s'%seq_list[ix]],amino_diam['%s'%seq_list[ix+1]]))
    else:
        pos.append([ pos[ix-1][0] + hbl + amino_diam['%s'%seq_list[ix]]/2. ,0,0])
pos=np.array(pos)
u=-1

for p in system.particles:
    u+=1
    if(u>=177):
        break
    p.position = pos[u]
    
###----------Make Harmonic Bonds----------###
harm = md.bond.harmonic()
for i in range(len(BT)):
    harm.bond_coeff.set(BT[i],k=3680,r0=BLDic[BT[i]])
    #harm.bond_coeff.set(BT[i],k=3680,r0=BLDic[BT[i]])



###----------Make Angles----------###
###Work in progress, make dict of angles. Look at particle types to deternine the right angle to use. 
    ###Get types: type[i-3],type[i-2],type[i-1]
    ### 
# h=3
# while(h<=(nparticles-1)):
#     system.angles.add("angleA", h-3, h-2, h-1)
#     h+=1
# harmonic = hoomd.md.angle.harmonic()
# harmonic.angle_coeff.set('angleA', k=50.0, t0=Angle(120))

nl = md.nlist.cell(r_buff = 0.2, check_period = 1)
#nl.tune(warmup=200000, r_min=0.05, r_max=1.0, jumps=20, steps=5000, set_max_check_period=False, quiet=False)
yukawa = md.pair.yukawa(r_cut=3.80, nlist=nl)
lj = md.pair.lj(r_cut=rcut, nlist=nl)


##----------Generates LJ pairs----------##
lj_debug = []
for i in range(0,len(sig)):
    for j in range(i,len(sig)):
        lj.pair_coeff.set(types[i],types[j],
                epsilon=0.5*(eps[types[i]]+eps[types[j]]),
                sigma=0.05*(sig[types[i]]+sig[types[j]]))
        lj_debug.append([types[i],types[j],
                'epsilon={0}'.format((eps[types[i]]+eps[types[j]])),
                'sigma={0}'.format((sig[types[i]]+sig[types[j]]))])
DB.DebugMessage("DEBUG: lj_debug = ", lj_debug, debug = False)
##----------Generates Yukawa pairs----------##
# for i in range(0,len(yukkap)):
#     for j in range(i,len(yukkap)):
#         yukawa.pair_coeff.set(types[i],types[j],
#                 epsilon=(yukeps[types[i]]+yukeps[types[j]]),
#                 kappa=(yukkap[types[i]]+yukkap[types[j]]))

##8.98755e9*1e9*((1.6e-19)/2.)**2*6.02e23/1000./80.
yukawa_debug = []
for i in range(0,len(types)):
    for j in range(i,len(types)):
        yukawa.pair_coeff.set(types[i],types[j],
                epsilon=(8.98755e9*1e9*amino_charge[types[i]]*(1.6e-19)*amino_charge[types[j]]*(1.6e-19)*6.02e23/1000./80.),
                kappa=(1.0), r_cut=3.5)
        if((8.98755e9*1e9*amino_charge[types[i]]*(1.6e-19)*amino_charge[types[j]]*(1.6e-19)*6.02e23/1000./80.)!=0):
            yukawa_debug.append([types[i],types[j],
                    'epsilon={0}'.format((8.98755e9*1e9*amino_charge[types[i]]*(1.6e-19)*amino_charge[types[j]]*(1.6e-19)*6.02e23/1000./80.)),
                    'kappa={0}'.format(1.0), 'r_cut={0}'.format(3.5)])
DB.DebugMessage("DEBUG: yukawa_debug = ", yukawa_debug, debug = False)


#system.replicate(nx=1,ny=1,nz=1)
hoomd.md.integrate.mode_standard(dt=0.002)
#hoomd.md.integrate.langevin(group=hoomd.group.all(), kT=0.1, seed = 5)
# hoomd.md.integrate.langevin(group=nonrigid, kT=kTinput, seed = SEED)
# hoomd.md.integrate.langevin(group=center, kT=kTinput, seed = SEED)
integrator_nonrigid = hoomd.md.integrate.langevin(group=nonrigid, kT=kTinput, seed = SEED)
integrator_center = hoomd.md.integrate.langevin(group=center, kT=kTinput, seed = SEED)
gamma=0.01
for i in range(len(seq_string)):
    integrator_nonrigid.set_gamma(seq_string[i], gamma=amino_masses[seq_string[i]]*gamma)
    integrator_center.set_gamma(seq_string[i], gamma=amino_masses[seq_string[i]]*gamma)
# hoomd.analyze.log(filename='LOG {0}.log'.format(t),quantities=['time','num_particles','ndof','translational_ndof','rotational_ndof',
#                                                                'potential_energy','kinetic_energy','translational_kinetic_energy',
#                                                                'rotational_kinetic_energy','temperature','pressure','pair_lj_energy',
#                                                                'pair_yukawa_energy','bond_harmonic_energy','angle_harmonic_energy'],period=1000,header_prefix='#' , overwrite=True)
hoomd.analyze.log(filename='LOG {0}.log'.format(t),quantities=['time','num_particles','ndof','translational_ndof','rotational_ndof',
                                                               'potential_energy','kinetic_energy','translational_kinetic_energy',
                                                               'rotational_kinetic_energy','temperature','pressure','pair_lj_energy',
                                                               'pair_yukawa_energy','bond_harmonic_energy'],period=1000,header_prefix='#' , overwrite=True)
hoomd.dump.gsd(filename='{0} {1}.gsd'.format(filename, t),period=1000, group = hoomd.group.all() ,overwrite = True, dynamic=['attribute','property','momentum','topology'])
hoomd.run(1e6)


Debug Message Format Is As Follows:
 DEBUG: Function: Parameter = Value at line <linenumber>
notice(2): Group "all" created containing 177 particles
DEBUG: types =  Particle types: ['M', 'r', 'A', 'Q', 'L', 'I', 'G', 'T', 'E', 'P', 'S', 'f', 'V', 'N', 'D', 'W', '1', '2', '3', '4']  at line  314
DEBUG: sig =  {'M': 6.18, 'r': 6.56, 'A': 5.04, 'Q': 6.02, 'L': 6.18, 'I': 6.18, 'G': 4.5, 'T': 5.62, 'E': 5.92, 'P': 5.56, 'S': 5.18, 'f': 6.36, 'V': 5.86, 'N': 5.68, 'D': 5.58, 'W': 6.78, '1': 1.6628919599999998, '2': 1.6628919599999998, '3': 1.6628919599999998, '4': 1.6628919599999998}  at line  326
DEBUG: eps =  {'M': 0.1, 'r': 0.1, 'A': 0.1, 'Q': 0.1, 'L': 0.1, 'I': 0.1, 'G': 0.1, 'T': 0.1, 'E': 0.1, 'P': 0.1, 'S': 0.1, 'f': 0.1, 'V': 0.1, 'N': 0.1, 'D': 0.1, 'W': 0.1, '1': 0.5, '2': 0.5, '3': 0.5, '4': 0.5}  at line  328
notice(2): constrain.rigid(): Creating 9 rigid bodies (adding 18 particles)
notice(2): Group "rigid_center" created containing 9 particles
notice(2): Group "nonrigid" crea

*Warning*: Integrator #0: Anisotropic integration requested, but no rotational degrees of freedom found for its group


Time 00:00:10 | Step 159012 / 1000000 | TPS 15901.1 | ETA 00:00:52
Time 00:00:20 | Step 317071 / 1000000 | TPS 15805.8 | ETA 00:00:43
Time 00:00:30 | Step 475324 / 1000000 | TPS 15825.3 | ETA 00:00:33
Time 00:00:40 | Step 630377 / 1000000 | TPS 15505.2 | ETA 00:00:23
Time 00:00:50 | Step 790522 / 1000000 | TPS 16014.5 | ETA 00:00:13
Time 00:01:00 | Step 952326 / 1000000 | TPS 16180.3 | ETA 00:00:02
Time 00:01:03 | Step 1000000 / 1000000 | TPS 16101.3 | ETA 00:00:00
Average TPS: 15882.5
---------
-- Neighborlist stats:
223848 normal updates / 10000 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 4 / n_neigh_avg: 0.0974359
shortest rebuild period: 2
-- Cell list stats:
Dimension: 810, 4, 4
n_min    : 0 / n_max: 4 / n_avg: 0.0150463
** run complete **


In [21]:
###Just so the workspace isn't so cramped. 

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [12]:
print(yukawa_debug)

[['r', 'r', 'epsilon=1.7313616319999998', 'kappa=1.0', 'r_cut=3.5'], ['r', 'E', 'epsilon=-1.7313616319999998', 'kappa=1.0', 'r_cut=3.5'], ['r', 'D', 'epsilon=-1.7313616319999998', 'kappa=1.0', 'r_cut=3.5'], ['E', 'E', 'epsilon=1.7313616319999998', 'kappa=1.0', 'r_cut=3.5'], ['E', 'D', 'epsilon=1.7313616319999998', 'kappa=1.0', 'r_cut=3.5'], ['D', 'D', 'epsilon=1.7313616319999998', 'kappa=1.0', 'r_cut=3.5']]


## LJ with amino diams of 1.0

In [21]:
import hoomd,imp
import hoomd
from hoomd import *
from hoomd import md
import numpy as np
import gsd.hoomd
import time 
import matplotlib.pyplot as plt
from decimal import *
from inspect import currentframe
import string

# class SimBox(self):
class SimBox():
#   def __init__(self, boxsize,nparticles):
#        self.boxsize=boxsize
#        self.nparticles=nparticles
        
    
    ###Creates Initial Snapshot
    def MakeSnapshot(self, boxsize, nparticles): 
        
        snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(L=boxsize+nparticles),
                                          bond_types=['tether'],particle_types=self.GenParticleTypes(nparticles, debug=True))
        return snapshot
    
    
    #Only used for initializing the system, do not use outside of MakeSnapshot!
    def GenParticleTypes(self, nparticles, debug=None):
        counter=0
        #List that will contain the particle types. Enable debug to print.
        GPT = [] 
        for k in range(nparticles):
            counter=k
            GPT.append('C%d'%counter)
        if(debug==True):
            print("DEBUG: GenParticleTypes: Particle Types = ",GPT)
        return GPT

class Polymer:

    def SetPolyProperties(self, nparticles, boxsize, diam=None, debug=None):
        if(diam==None):
            diam=1.0 #default diameter
        for i in range(nparticles):
            system.particles.get(i).position = [-(boxsize-nparticles)+(((boxsize-nparticles))/nparticles)*i,0,0]
            system.particles.get(i).diameter = 1.0
            #system.particles.get(i).type = ('C%d'%(i))
            if(debug==True):
                print("DEBUG: SetPolyProperties: Particle %d is at "%i, [nparticles+i,0,0])
    
    #MakePolyTage is just a debugging tool, ignore. 
    def MakePolyTags(self, nparticles, debug=None):
        P_Tag_List = []
        for i in range(nparticles):
            system.particles.get(i).type=('C%d'%(i+1))
            P_Tag_List.append('C%d'%(i+1))
        if(debug==True):
            print("DEBUG: SetPolyProperties: P_Tag_List is ", P_Tag_List)
        
    def PatchTypes(self, tot_npatch, debug=None):
        patch_types = []
        for i in range(tot_npatch):
            system.particles.types.add('%d'%(i+1))
            patch_types.append('%d'%(i+1))
        if(debug==True):
            print("DEBUG: PatchTypes: ",patch_types)
        return patch_types

            
    
    def MakeBonds(self, nparticles, debug=None):
        DebugBonds = []
        f=0
        for f in range(nparticles-1):
            system.bonds.add('tether',f,f+1)
            DebugBonds.append([f,f+1])
            f+=1
        if(debug==True):
            print("DEBUG: MakeBonds: Bonds = ", DebugBonds)  
class Debug:
    def PrintAllParticleTags(self, nparticles, tot_npatch):
        X = []
        for i in range(nparticles+tot_npatch):
            X.append(system.particles[i].tag)
        print("DEBUG: List of All Particle Tags: ", X)
    def PrintSysParticles(self, debug=None):
        if(debug==True):
            for i in range(len(system.particles)):
                print(system.particles[i])
    def PrintLJPairs(self, sig, types):
        for i in range(0,len(sig)):
            for j in range(i,len(sig)):
                print([types[i],types[j]])
    def DebugMessage(self, string, variable, debug=None):
        if(debug==True):
            cf = currentframe()
            print("%s"%string, variable, " at line ", cf.f_back.f_lineno)
    def DebugInfo(self):
        print("Debug Message Format Is As Follows:\n DEBUG: Function: Parameter = Value at line <linenumber>")

class MomentOfInertia:
  
    #Principle Moment Calculator
    #added an option to return the inertia tensor for debugging
    def Moment(self, coord, Mass, return_tensor = None, debug=None):
        DB = Debug()
        #Referencing local variables
        cx,cy,cz = 0.0,0.0,0.0
        x,y,z = 0.0,0.0,0.0
        cXYZ = []
        Ixx,Ixy,Ixz,Iyy,Iyz,Izz = 0.0,0.0,0.0,0.0,0.0,0.0
        TotalMass = sum(Mass)
        if(debug==True):
            print("DEBUG: Moment: len(coord)= ", len(coord))
            print("DEBUG: Moment: TotalMass = ", TotalMass)
        for i in range(len(Mass)):
                x += Mass[i]*coord[i][0]
                y += Mass[i]*coord[i][1]
                z += Mass[i]*coord[i][2]
                if(debug==True):
                    print("DEBUG: Moment: x = Mass[%d]"%i +"*coord[%d][0]"%i +" =%f "%x)
                    print("DEBUG: Moment: y = Mass[%d]"%i +"*coord[%d][1]"%i +" =%f "%y)
                    print("DEBUG: Moment: z = Mass[%d]"%i +"*coord[%d][2]"%i +" =%f "%z)
        cx = x/TotalMass
        cy = y/TotalMass
        cz = z/TotalMass
        if(debug==True):
            print("DEBUG: Moment: cx = x/TotalMass = ", cx)
            print("DEBUG: Moment: cy = y/TotalMass = ", cy)
            print("DEBUG: Moment: cz = y/TotalMass = ", cz)
        com = [cx,cy,cz]
        for i in range(len(coord)):
            cXYZ.append([coord[i][0]-com[0], coord[i][1]-com[1], coord[i][2]-com[2]]) 
        if(debug==True):
            print("DEBUG: Moment: cXYZ = ", cXYZ)
            print("DEBUG: Moment: len(cXYZ) = ", len(cXYZ))
        #Constructs Inertia Tensor
        #Useful Property: Ixy=Iyx, Ixz=Izx, Iyz=Izy
        for i in range(len(coord)):
            Ixx += Mass[i]*(cXYZ[i][1]**2 + cXYZ[i][2]**2)
            Iyy += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][2]**2)
            Izz += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][1]**2)
            Ixy += -Mass[i]*cXYZ[i][0]*cXYZ[i][1]
            Ixz += -Mass[i]*cXYZ[i][0]*cXYZ[i][2]
            Iyz += -Mass[i]*cXYZ[i][1]*cXYZ[i][2]
        if(debug==True):
            print("DEBUG: Moment: Ixx = ", Ixx)
            print("DEBUG: Moment: Iyy = ", Iyy)
            print("DEBUG: Moment: Izz = ", Izz)
            print("DEBUG: Moment: Ixy = ", Ixy)
            print("DEBUG: Moment: Ixz = ", Ixz)
            print("DEBUG: Moment: Iyz = ", Iyz)

        #Computes eignevalues, finds principle moments
        Imatrix = np.matrix([[Ixx,Ixy,Ixz],[Ixy,Iyy,Iyz],[Ixz,Iyz,Izz]])
        Idiag = np.linalg.eig(Imatrix)
        if(debug==True):
            print("DEBUG: Moment: Idiag = ",Idiag)
            print("DEBUG: Moment: Idiag[0] = ",Idiag[0])
        comX = 0-com[0]
        comY = 0-com[1] 
        for i in range(len(coord)):
            coord[i] = [coord[i][0]+comX,coord[i][1]+comY,coord[i][2]]
        if(return_tensor==True):
            return Idiag[0], Imatrix
        else:
            return Idiag[0]
#Just for flexability 
def Angle(angle):
    angle = angle*np.pi/180
    return angle
DB = Debug()
DB.DebugInfo()
SB = SimBox()
MoI = MomentOfInertia()
Poly = Polymer()
#Filename (date and time will be appended onto the file name)
filename = "UBQLN2"
#Box Size
boxsize = 5
#Number of constituent particles
nparticles = 177 #will need to be 1
#Total number of patches in sim
#tot_npatch = 79
tot_npatch = 4
#Rigid Diameter
diam = 1.0
#List of Patch Diameters
pdiam = [1.0]
#LJ Cutoff
rcut = 2.5
#Random Seed
SEED = np.random.randint(0,100000000)
#Empty list for setting up sig values
Sigma = []
#Empty list for setting up eps values
Epsilon = []
#Empty list for yukawa kappa
YukKap = []
#Empty list for yukawa eps
YukEps = []
#Allows me to make unique file names
t = time.strftime(time.strftime("%d %b %H:%M:%S", time.gmtime()))
#Harmonic Bond Length
hbl = .380
#Temperature/ kT value
Temperature=200
kTinput=Temperature * 8.3144598/1000.
##----------SETUP----------##
context.initialize("")
amino_masses = dict([['A',89.094],['C',121.150],['D',133.103],['E',147.130],['F',165.192],['G',75.067],
                    ['H',155.157],['I',131.175],['K',146.19],['L',131.175],['M',149.210],['N',132.119],
                    ['P',115.132],['Q',146.146],['R',174.204],['S',105.093],['T',119.120],['V',117.148],
                    ['W',204.229],['Y',181.191],
                    ['a',89.094],['c',121.150],['d',133.103],['e',147.130],['f',165.192],['g',75.067],
                    ['h',155.157],['i',131.175],['k',146.19],['l',131.175],['m',149.210],['n',132.119],
                    ['p',115.132],['q',146.146],['r',174.204],['s',105.093],['t',119.120],['v',117.148],
                    ['w',204.229],['y',181.191]])

#From source [4]
amino_diam = dict([['A',5.04],['C',5.48],['D',5.58],['E',5.92],['F',6.36],['G',4.50],
                    ['H',6.08],['I',6.18],['K',6.36],['L',6.18],['M',6.18],['N',5.68],
                    ['P',5.56],['Q',6.02],['R',6.56],['S',5.18],['T',5.62],['V',5.86],
                    ['W',6.78],['Y',6.46],['a',5.04],['c',5.48],['d',5.58],['e',5.92],['f',6.36],['g',4.50],
                    ['h',6.08],['i',6.18],['k',6.36],['l',6.18],['m',6.18],['n',5.68],
                    ['p',5.56],['q',6.02],['r',6.56],['s',5.18],['t',5.62],['v',5.86],
                    ['w',6.78],['y',6.46]])
# amino_diam = dict([['A',0.504],['C',0.548],['D',0.558],['E',0.592],['F',0.636],['G',0.450],
#                     ['H',0.608],['I',0.618],['K',0.636],['L',0.618],['M',0.618],['N',0.568],
#                     ['P',0.556],['Q',0.602],['R',0.656],['S',0.518],['T',0.562],['V',0.586],
#                     ['W',0.678],['Y',0.646],['a',0.504],['c',0.548],['d',0.558],['e',0.592],['f',0.636],['g',0.450],
#                     ['h',0.608],['i',0.618],['k',0.636],['l',0.618],['m',0.618],['n',0.568],
#                     ['p',0.556],['q',0.602],['r',0.656],['s',0.518],['t',0.562],['v',0.586],
#                     ['w',0.678],['y',0.646]])
amino_charge = dict([['A',0.0],['C',0.0],['D',-1.0],['E',-1.0],['F',0.0],['G',0.0],
                    ['H',0.5],['I',0.0],['K',1.0],['L',0.0],['M',0.0],['N',0.0],
                    ['P',0.0],['Q',0.0],['R',1.0],['S',0.0],['T',0.0],['V',0.0],
                    ['W',0.0],['Y',0.0],['a',0.0],['c',0.0],['d',-1.0],['e',-1.0],['f',0.0],['g',0.0],
                    ['h',0.5],['i',0.0],['k',1.0],['l',0.0],['m',0.0],['n',0.0],
                    ['p',0.0],['q',0.0],['r',1.0],['s',0.0],['t',0.0],['v',0.0],
                    ['w',0.0],['y',0.0]])

#Creates dict for patch charges
pcharge_dict = []
for i in range(tot_npatch):
    pcharge_dict.append(['{0}'.format(i+1), 0.0])
pcharge_dict = dict(pcharge_dict)

#Appends pcharge_dict to amino_charge
amino_charge.update(pcharge_dict)

##Generates all bond types and bond lengths between pairs of amino acids
seq_string = "ACDEFGHIKLMNPQRSTVWYacdefghiklmnpqrstvwy" #List of all amino acids in sim
BL = [] #Bond Lengths
BT = [] #Bond Types
for i in range(len(seq_string)):
    #for j in range(i,len(seq_string)):
    for j in range(len(seq_string)):
        BL.append(['{0}{1}'.format(seq_string[i],seq_string[j]), amino_diam[seq_string[i]]/2. + amino_diam[seq_string[j]]/2. + hbl])
        BT.append('{0}{1}'.format(seq_string[i],seq_string[j]))
BLDic = dict(BL) #Bond Length Dictionary

#Complete Sequence
#sequence = 'MRAMQalmqiqqglqtlateapglipsftpgvgvgvlgtaigpvgpvtpiGPIgpivpftPIGPIGPIGPTGPAAPPGSTGSGGPTGPTVSSAAPSETTSPTSESGPNqqfiqqmvqalaganaPQLPNPEVRFQQQLEQLNangflNREANLQALIATGGDINAAIErllgsqPSW'      
sequence = 'MrAMQALMQIQQGLQTLATEAPGLIPSfTPGVGVGVLGTAIGPVGPVTPIGPIGPIVPfTPIGPIGPIGPTGPAAPPGSTGSGGPTGPTVSSAAPSETTSPTSESGPNQQfIQQMVQALAGANAPQLPNPEVrfQQQLEQLNANGfLNrEANLQALIATGGDINAAIErLLGSQPSW'
seq_list = []
rigid_tags = []
for i in range(len(sequence)):
    seq_list.append('%s'%sequence[i])
for i in range(len(sequence)):
    for j in range(26):
        if(sequence[i] is string.ascii_lowercase[j]):
            rigid_tags.append(i)
def unique(list1): #from https://www.geeksforgeeks.org/python-get-unique-values-list/
    # intilize a null list 
    unique_list = [] 
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list


#Creates a list of all amino acid types in polymer
seq_types = unique(seq_list)



#Creates snapshot, initializes particle types
# snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(Lx=3000, Ly=27, Lz=27),
#         bond_types=['tether'],particle_types=seq_types)
snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(Lx=250, Ly=15, Lz=15),
        bond_types=BT,particle_types=seq_types)
system = hoomd.init.read_snapshot(snapshot)

#Sets each particle's type equal to it's amino acid
for i in range(nparticles):
    system.particles.get(i).type = ('%s'%seq_list[i])
    system.particles.get(i).diameter = 1.0

##Adds patch Patch Types from [1,tot_npatch]
##----------Assign Masses----------##
ix=0
for p in system.particles:
    p.mass = amino_masses[p.type]
    p.charge = amino_charge[p.type]

    
    
patch_types = Poly.PatchTypes(tot_npatch, debug=False) 
types = system.particles.types
DB.DebugMessage("DEBUG: types = ", types, debug=True)

patch_sigma = kTinput
##----------Builds Dictionaries for LJ Parameters---------##
for i in range(len(types)):
    if(i>=(len(types)-tot_npatch)):
        Sigma.append(('%s'%types[i], patch_sigma))
        Epsilon.append(('%s'%types[i], 0.5)) 
    elif(i<len(types)):
        Sigma.append(('%s'%types[i], amino_diam[types[i]]))
        Epsilon.append(('%s'%types[i], 0.1))  
sig = dict(Sigma)
DB.DebugMessage("DEBUG: sig = ", sig, debug = True)
eps = dict(Epsilon)
DB.DebugMessage("DEBUG: eps = ", eps, debug = True)


##----------Builds Dictionaries for Yukawa Parameters---------##
    ##Allows independent assigning of parameters of patches and particles
# for i in range(len(types)):
#     if(i>=(len(types)-tot_npatch)): #i=npaticles is where the indices for the patch tags start
#         YukKap.append(('%s'%types[i], 0.0))
#         YukEps.append(('%s'%types[i], 0.0)) 
#     elif(i<len(types)):
#         YukKap.append(('%s'%types[i], 0.0))
#         YukEps.append(('%s'%types[i], 0.0)) 
# yukkap = dict(YukKap)
# DB.DebugMessage("DEBUG: yukkap = ", yukkap, debug = True)
# yukeps = dict(YukEps)
# DB.DebugMessage("DEBUG: yukeps = ", yukeps, debug = True)

###----------Make Rigid Bodies----------###
rigid=hoomd.md.constrain.rigid()
coord_patch_group_a = [[2.52*np.cos(Angle(60)),0,2.52*np.sin(Angle(60))],
                       [2.52*np.cos(Angle(60)),0,-2.52*np.sin(Angle(60))]]

coord_patch_group_l = [[3.09*np.cos(Angle(60)),0,3.09*np.sin(Angle(60))],
                       [3.09*np.cos(Angle(60)),0,-3.09*np.sin(Angle(60))]]

coord_patch_group_m = [[3.09*np.cos(Angle(60)),0,3.09*np.sin(Angle(60))],
                       [3.09*np.cos(Angle(60)),0,-3.09*np.sin(Angle(60))]]

coord_patch_group_q = [[3.01*np.cos(Angle(60)),0,3.01*np.sin(Angle(60))],
                       [3.01*np.cos(Angle(60)),0,-3.01*np.sin(Angle(60))]]

coord_patch_group_k = [[6.36*np.cos(Angle(60)),0,6.36*np.sin(Angle(60))],
                       [6.36*np.cos(Angle(60)),0,-6.36*np.sin(Angle(60))]]

coord_patch_group_i = [[3.09*np.cos(Angle(60)),0,3.09*np.sin(Angle(60))],
                       [3.09*np.cos(Angle(60)),0,-3.09*np.sin(Angle(60))]]

coord_patch_group_g = [[2.25*np.cos(Angle(60)),0,2.25*np.sin(Angle(60))],
                       [2.25*np.cos(Angle(60)),0,-2.25*np.sin(Angle(60))]]

coord_patch_group_t = [[2.81*np.cos(Angle(60)),0,2.81*np.sin(Angle(60))],
                       [2.81*np.cos(Angle(60)),0,-2.81*np.sin(Angle(60))]]

coord_patch_group_e = [[2.96*np.cos(Angle(60)),0,2.96*np.sin(Angle(60))],
                       [2.96*np.cos(Angle(60)),0,-2.96*np.sin(Angle(60))]]

coord_patch_group_p = [[2.78*np.cos(Angle(60)),0,2.78*np.sin(Angle(60))],
                       [2.78*np.cos(Angle(60)),0,-2.78*np.sin(Angle(60))]]

coord_patch_group_s = [[2.59*np.cos(Angle(60)),0,2.59*np.sin(Angle(60))],
                       [2.59*np.cos(Angle(60)),0,-2.59*np.sin(Angle(60))]]

coord_patch_group_f = [[0.5*np.cos(Angle(60)),0,0.5*np.sin(Angle(60))],
                       [0.5*np.cos(Angle(60)),0,0.5*np.sin(Angle(60))]]

coord_patch_group_v = [[2.93*np.cos(Angle(60)),0,2.93*np.sin(Angle(60))],
                       [2.93*np.cos(Angle(60)),0,-2.93*np.sin(Angle(60))]]

coord_patch_group_n = [[2.84*np.cos(Angle(60)),0,2.84*np.sin(Angle(60))],
                       [2.84*np.cos(Angle(60)),0,-2.84*np.sin(Angle(60))]]

coord_patch_group_r = [[0.5*np.cos(Angle(60)),0,0.5*np.sin(Angle(60))],
                       [0.5*np.cos(Angle(60))/10.,0,0.5*np.sin(Angle(60))]]

rigid_neigh_1 =[[0.1*np.cos(Angle(60)),0,0.1*np.sin(Angle(60))],
                [0.1*np.cos(Angle(60)),0,-0.1*np.sin(Angle(60))],
                [-hbl,0,0],[hbl,0,0]]

# mass_patch_group_1 = [0.5,0.5,75.067,89.094]
mass_patch_groups = []
#reference rigid_tags for indices
for i in range(len(sequence)):
    for j in range(26):
        if(sequence[i] is string.ascii_lowercase[j]):
            if((i+1)>=len(sequence)):
                mass_patch_groups.append([0.5,0.5,amino_masses[sequence[i-1]]])
            elif((i+1)<len(sequence) and (i-1)>=0):
                
                mass_patch_groups.append([0.5,0.5,amino_masses[sequence[i-1]],amino_masses[sequence[i+1]]])
                
            elif((i-1)<0):
                mass_patch_groups.append([0.5,0.5,amino_masses[sequence[i+1]]])
diameters_patch = [0.2,0.2]

eigen_groups = []
for i in range(len(rigid_tags)):
    eigen_groups.append(MoI.Moment(rigid_neigh_1, mass_patch_groups[i], debug=False))


# rigid.set_param('a',positions=coord_patch_group_a, types=['1','2'], diameters = diameters_patch)
# rigid.set_param('l',positions=coord_patch_group_l, types=['3','4'], diameters = diameters_patch)
# rigid.set_param('m',positions=coord_patch_group_m, types=['5','6'], diameters = diameters_patch)
# rigid.set_param('q',positions=coord_patch_group_q, types=['7','8'], diameters = diameters_patch)
# rigid.set_param('i',positions=coord_patch_group_i, types=['9','10'], diameters = diameters_patch)
# rigid.set_param('g',positions=coord_patch_group_g, types=['11','12'], diameters = diameters_patch)
# rigid.set_param('t',positions=coord_patch_group_t, types=['13','14'], diameters = diameters_patch)
# rigid.set_param('e',positions=coord_patch_group_e, types=['15','16'], diameters = diameters_patch)
# rigid.set_param('p',positions=coord_patch_group_p, types=['17','18'], diameters = diameters_patch)
# rigid.set_param('s',positions=coord_patch_group_s, types=['19','20'], diameters = diameters_patch)
rigid.set_param('f',positions=coord_patch_group_f, types=['1','2'], diameters = diameters_patch)
# rigid.set_param('v',positions=coord_patch_group_v, types=['23','24'], diameters = diameters_patch)
# rigid.set_param('n',positions=coord_patch_group_n, types=['25','26'], diameters = diameters_patch)
rigid.set_param('r',positions=coord_patch_group_r, types=['3','4'], diameters = diameters_patch)
#rigid.set_param('k',positions=coord_patch_group_f, types=['3','4'], diameters = diameters_patch)
rigid.create_bodies()

# system.particles[191].charge = 1.00
# system.particles[192].charge = 1.00


center = hoomd.group.rigid_center()
nonrigid = hoomd.group.nonrigid()
part = hoomd.group.all()
gpoly = hoomd.group.union(name='gpoly', a=center, b=nonrigid) 
patches = hoomd.group.difference(name = 'patches', a=part, b=gpoly)
for p in patches:
    p.mass = 0.5
for i in range(len(rigid_tags)):
    Z = rigid_tags[i]
    system.particles[Z].moment_inertia = eigen_groups[i]

##----------Make Bonds----------##
bi = 0 #bond index
DebugBonds = []
for bi in range(nparticles-1):
    system.bonds.add('{0}{1}'.format(sequence[bi],sequence[bi+1]),bi,bi+1)
    #system.bonds.add('tether',bi,bi+1)
    DebugBonds.append([bi,bi+1])
    bi+=1

pos = []
# for ix in range(nparticles):
#     if(ix<nparticles-1):
#         if(ix!=0):
#             pos.append([pos[ix-1][0] + hbl + amino_diam['%s'%seq_list[ix]]/2. +amino_diam['%s'%seq_list[ix+1]]/2. ,0,0])
#             #print("diams = {0} {1} {2}".format(hbl,amino_diam['%s'%seq_list[ix]],amino_diam['%s'%seq_list[ix+1]]))
#         elif(ix==0):
#             pos.append([-500 + hbl + amino_diam['%s'%seq_list[ix]]/2. +amino_diam['%s'%seq_list[ix+1]]/2. ,0,0])
#             #print("diams = {0} {1} {2}".format(hbl,amino_diam['%s'%seq_list[ix]],amino_diam['%s'%seq_list[ix+1]]))
#     else:
#         pos.append([ pos[ix-1][0] + hbl + amino_diam['%s'%seq_list[ix]]/2. ,0,0])
for ix in range(nparticles):
    if(ix<nparticles-1):
        if(ix!=0):
            pos.append([pos[ix-1][0] + hbl + 0.5 ,0,0])
            #print("diams = {0} {1} {2}".format(hbl,amino_diam['%s'%seq_list[ix]],amino_diam['%s'%seq_list[ix+1]]))
        elif(ix==0):
            pos.append([-122 + hbl + 0.5 ,0,0])
            #print("diams = {0} {1} {2}".format(hbl,amino_diam['%s'%seq_list[ix]],amino_diam['%s'%seq_list[ix+1]]))
    else:
        pos.append([ pos[ix-1][0] + hbl + 0.5 ,0,0])
pos=np.array(pos)
u=-1

for p in system.particles:
    u+=1
    if(u>=177):
        break
    p.position = pos[u]
    
###----------Make Harmonic Bonds----------###
harm = md.bond.harmonic()
for i in range(len(BT)):
    harm.bond_coeff.set(BT[i],k=3680,r0=0.380)
    #harm.bond_coeff.set(BT[i],k=3680,r0=BLDic[BT[i]])



###----------Make Angles----------###
###Work in progress, make dict of angles. Look at particle types to deternine the right angle to use. 
    ###Get types: type[i-3],type[i-2],type[i-1]
    ### 
# h=3
# while(h<=(nparticles-1)):
#     system.angles.add("angleA", h-3, h-2, h-1)
#     h+=1
# harmonic = hoomd.md.angle.harmonic()
# harmonic.angle_coeff.set('angleA', k=50.0, t0=Angle(120))

nl = md.nlist.cell(r_buff = 0.2, check_period = 1)
#nl.tune(warmup=200000, r_min=0.05, r_max=1.0, jumps=20, steps=5000, set_max_check_period=False, quiet=False)
yukawa = md.pair.yukawa(r_cut=3.80, nlist=nl)
lj = md.pair.lj(r_cut=rcut, nlist=nl)


##----------Generates LJ pairs----------##
lj_debug = []
for i in range(0,len(sig)):
    for j in range(i,len(sig)):
        lj.pair_coeff.set(types[i],types[j],
                epsilon=0.5*(eps[types[i]]+eps[types[j]]),
                sigma=0.05*(sig[types[i]]+sig[types[j]]))
        lj_debug.append([types[i],types[j],
                'epsilon={0}'.format((eps[types[i]]+eps[types[j]])),
                'sigma={0}'.format((sig[types[i]]+sig[types[j]]))])
DB.DebugMessage("DEBUG: lj_debug = ", lj_debug, debug = False)
##----------Generates Yukawa pairs----------##
# for i in range(0,len(yukkap)):
#     for j in range(i,len(yukkap)):
#         yukawa.pair_coeff.set(types[i],types[j],
#                 epsilon=(yukeps[types[i]]+yukeps[types[j]]),
#                 kappa=(yukkap[types[i]]+yukkap[types[j]]))

##8.98755e9*1e9*((1.6e-19)/2.)**2*6.02e23/1000./80.
yukawa_debug = []
for i in range(0,len(types)):
    for j in range(i,len(types)):
        yukawa.pair_coeff.set(types[i],types[j],
                epsilon=(8.98755e9*1e9*amino_charge[types[i]]*(1.6e-19)*amino_charge[types[j]]*(1.6e-19)*6.02e23/1000./80.),
                kappa=(1.0), r_cut=3.5)
        if((8.98755e9*1e9*amino_charge[types[i]]*(1.6e-19)*amino_charge[types[j]]*(1.6e-19)*6.02e23/1000./80.)!=0):
            yukawa_debug.append([types[i],types[j],
                    'epsilon={0}'.format((8.98755e9*1e9*amino_charge[types[i]]*(1.6e-19)*amino_charge[types[j]]*(1.6e-19)*6.02e23/1000./80.)),
                    'kappa={0}'.format(1.0), 'r_cut={0}'.format(3.5)])
DB.DebugMessage("DEBUG: yukawa_debug = ", yukawa_debug, debug = False)


#system.replicate(nx=1,ny=1,nz=1)
hoomd.md.integrate.mode_standard(dt=0.0001)
#hoomd.md.integrate.langevin(group=hoomd.group.all(), kT=0.1, seed = 5)
integrator_nonrigid = hoomd.md.integrate.langevin(group=nonrigid, kT=kTinput, seed = SEED)
integrator_center = hoomd.md.integrate.langevin(group=center, kT=kTinput, seed = SEED)
gamma=0.01
for i in range(len(seq_string)):
    integrator_nonrigid.set_gamma(seq_string[i], gamma=amino_masses[seq_string[i]]*gamma)
    integrator_center.set_gamma(seq_string[i], gamma=amino_masses[seq_string[i]]*gamma)
# hoomd.analyze.log(filename='LOG {0}.log'.format(t),quantities=['time','num_particles','ndof','translational_ndof','rotational_ndof',
#                                                                'potential_energy','kinetic_energy','translational_kinetic_energy',
#                                                                'rotational_kinetic_energy','temperature','pressure','pair_lj_energy',
#                                                                'pair_yukawa_energy','bond_harmonic_energy','angle_harmonic_energy'],period=1000,header_prefix='#' , overwrite=True)
hoomd.analyze.log(filename='LOG {0}.log'.format(t),quantities=['time','num_particles','ndof','translational_ndof','rotational_ndof',
                                                               'potential_energy','kinetic_energy','translational_kinetic_energy',
                                                               'rotational_kinetic_energy','temperature','pressure','pair_lj_energy',
                                                               'pair_yukawa_energy','bond_harmonic_energy'],period=1000,header_prefix='#' , overwrite=True)
hoomd.dump.gsd(filename='{0} {1}.gsd'.format(filename, t),period=1000, group = hoomd.group.all() ,overwrite = True, dynamic=['attribute','property','momentum','topology'])
hoomd.run(1e6)


Debug Message Format Is As Follows:
 DEBUG: Function: Parameter = Value at line <linenumber>
notice(2): Group "all" created containing 177 particles
DEBUG: types =  Particle types: ['M', 'r', 'A', 'Q', 'L', 'I', 'G', 'T', 'E', 'P', 'S', 'f', 'V', 'N', 'D', 'W', '1', '2', '3', '4']  at line  314
DEBUG: sig =  {'M': 6.18, 'r': 6.56, 'A': 5.04, 'Q': 6.02, 'L': 6.18, 'I': 6.18, 'G': 4.5, 'T': 5.62, 'E': 5.92, 'P': 5.56, 'S': 5.18, 'f': 6.36, 'V': 5.86, 'N': 5.68, 'D': 5.58, 'W': 6.78, '1': 1.6628919599999998, '2': 1.6628919599999998, '3': 1.6628919599999998, '4': 1.6628919599999998}  at line  326
DEBUG: eps =  {'M': 0.1, 'r': 0.1, 'A': 0.1, 'Q': 0.1, 'L': 0.1, 'I': 0.1, 'G': 0.1, 'T': 0.1, 'E': 0.1, 'P': 0.1, 'S': 0.1, 'f': 0.1, 'V': 0.1, 'N': 0.1, 'D': 0.1, 'W': 0.1, '1': 0.5, '2': 0.5, '3': 0.5, '4': 0.5}  at line  328
notice(2): constrain.rigid(): Creating 9 rigid bodies (adding 18 particles)
notice(2): Group "rigid_center" created containing 9 particles
notice(2): Group "nonrigid" crea

*Warning*: Integrator #0: Anisotropic integration requested, but no rotational degrees of freedom found for its group


Time 00:00:10 | Step 131689 / 1000000 | TPS 13168.9 | ETA 00:01:05
Time 00:00:20 | Step 224999 / 1000000 | TPS 9331 | ETA 00:01:23
Time 00:00:30 | Step 276914 / 1000000 | TPS 5191.45 | ETA 00:02:19
Time 00:00:40 | Step 330855 / 1000000 | TPS 5394.05 | ETA 00:02:04
Time 00:00:50 | Step 396298 / 1000000 | TPS 6544.21 | ETA 00:01:32
Time 00:01:00 | Step 454801 / 1000000 | TPS 5850.06 | ETA 00:01:33
Time 00:01:10 | Step 531527 / 1000000 | TPS 7672.6 | ETA 00:01:01
Time 00:01:20 | Step 615585 / 1000000 | TPS 8405.74 | ETA 00:00:45
Time 00:01:30 | Step 685953 / 1000000 | TPS 7036.73 | ETA 00:00:44
Time 00:01:40 | Step 748678 / 1000000 | TPS 6272.42 | ETA 00:00:40
Time 00:01:50 | Step 809493 / 1000000 | TPS 6081.49 | ETA 00:00:31
Time 00:02:00 | Step 875258 / 1000000 | TPS 6576.48 | ETA 00:00:18
Time 00:02:10 | Step 937837 / 1000000 | TPS 6257.84 | ETA 00:00:09
Time 00:02:20 | Step 991689 / 1000000 | TPS 5385.16 | ETA 00:00:01
Time 00:02:22 | Step 1000000 / 1000000 | TPS 5377.06 | ETA 00:00:0

In [4]:
for p in system.particles:
    print(p.position)

(-193.45704331277875, 0.0, -1075.039698637625)
(-522.3807196162916, 0.0, 5566.897774918932)
(9497.68901803105, 0.0, -9060.480393100697)
(-378.43523330007133, 0.0, -1.4191167170022434)
(-555.9355988431723, 0.0, 0.0)
(-494.6200066087503, 0.0, 0.0)
(-493.62003198170385, 0.0, 0.0)
(-561.1990776099979, 0.0, 0.0)
(-478.84770252411, 0.0, 0.0)
(-474.18198806599145, 0.0, 0.0)
(-429.946433285035, 0.0, 0.0)
(-488.6199617259453, 0.0, 0.0)
(-487.62000784365966, 0.0, 0.0)
(-454.59637156576395, 0.0, 0.0)
(-558.0658460610396, 0.0, 0.0)
(-434.1095278085645, 0.0, 0.0)
(-451.59631265209896, 0.0, 0.0)
(-493.8129390292333, 0.0, 0.0)
(-431.1095500516807, 0.0, 0.0)
(-498.42512553364503, 0.0, 0.0)
(-458.7736241008022, 0.0, 0.0)
(-495.0632832390647, 0.0, 0.0)
(-503.1263723425664, 0.0, 0.0)
(-476.620047130932, 0.0, 0.0)
(-497.2100209613371, 0.0, 0.0)
(-488.8038132706221, 0.0, -0.039712206955481305)
(-593.8622302258825, 0.0, -19.767732172899265)
(-38006.902999517544, 0.0, 25188.011934071)
(54036.31814478647, 0.0

In [19]:
print(pos)

[[-120.62    0.      0.  ]
 [-119.24    0.      0.  ]
 [-117.86    0.      0.  ]
 [-116.48    0.      0.  ]
 [-115.1     0.      0.  ]
 [-113.72    0.      0.  ]
 [-112.34    0.      0.  ]
 [-110.96    0.      0.  ]
 [-109.58    0.      0.  ]
 [-108.2     0.      0.  ]
 [-106.82    0.      0.  ]
 [-105.44    0.      0.  ]
 [-104.06    0.      0.  ]
 [-102.68    0.      0.  ]
 [-101.3     0.      0.  ]
 [ -99.92    0.      0.  ]
 [ -98.54    0.      0.  ]
 [ -97.16    0.      0.  ]
 [ -95.78    0.      0.  ]
 [ -94.4     0.      0.  ]
 [ -93.02    0.      0.  ]
 [ -91.64    0.      0.  ]
 [ -90.26    0.      0.  ]
 [ -88.88    0.      0.  ]
 [ -87.5     0.      0.  ]
 [ -86.12    0.      0.  ]
 [ -84.74    0.      0.  ]
 [ -83.36    0.      0.  ]
 [ -81.98    0.      0.  ]
 [ -80.6     0.      0.  ]
 [ -79.22    0.      0.  ]
 [ -77.84    0.      0.  ]
 [ -76.46    0.      0.  ]
 [ -75.08    0.      0.  ]
 [ -73.7     0.      0.  ]
 [ -72.32    0.      0.  ]
 [ -70.94    0.      0.  ]
 

In [6]:
Start=-500
pox=[]
for i in range(nparticles):
    #start+=(i+hbl)
    pox.append([Start+i+hbl+20.0,0,0])

pox=np.array(pox)
print(pox)

[[-479.62    0.      0.  ]
 [-478.62    0.      0.  ]
 [-477.62    0.      0.  ]
 [-476.62    0.      0.  ]
 [-475.62    0.      0.  ]
 [-474.62    0.      0.  ]
 [-473.62    0.      0.  ]
 [-472.62    0.      0.  ]
 [-471.62    0.      0.  ]
 [-470.62    0.      0.  ]
 [-469.62    0.      0.  ]
 [-468.62    0.      0.  ]
 [-467.62    0.      0.  ]
 [-466.62    0.      0.  ]
 [-465.62    0.      0.  ]
 [-464.62    0.      0.  ]
 [-463.62    0.      0.  ]
 [-462.62    0.      0.  ]
 [-461.62    0.      0.  ]
 [-460.62    0.      0.  ]
 [-459.62    0.      0.  ]
 [-458.62    0.      0.  ]
 [-457.62    0.      0.  ]
 [-456.62    0.      0.  ]
 [-455.62    0.      0.  ]
 [-454.62    0.      0.  ]
 [-453.62    0.      0.  ]
 [-452.62    0.      0.  ]
 [-451.62    0.      0.  ]
 [-450.62    0.      0.  ]
 [-449.62    0.      0.  ]
 [-448.62    0.      0.  ]
 [-447.62    0.      0.  ]
 [-446.62    0.      0.  ]
 [-445.62    0.      0.  ]
 [-444.62    0.      0.  ]
 [-443.62    0.      0.  ]
 